In [ ]:
import requests
import urllib.request as urllib2
import pickle
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from tqdm.notebook import tqdm

## Gathering Data (lyrics) - Source: genius.com

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("./chromedriver.exe", chrome_options=options)

c:\users\thien\.virtualenvs\rap_lyrics-ld_741gw\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [8]:
BASE_URL = "https://genius.com/"
URLS = [
    "https://genius.com/artists/Eminem",
    "https://genius.com/artists/J-cole",
    "https://genius.com/artists/Kanye-west",
    "https://genius.com/artists/2pac",
    "https://genius.com/artists/The-notorious-big",
    "https://genius.com/artists/Logic",
    "https://genius.com/artists/Nas",
    "https://genius.com/artists/Joyner-lucas",
    "https://genius.com/artists/Juice-wrld",
    "https://genius.com/artists/Lil-pump",
    "https://genius.com/artists/Nicki-minaj",
    "https://genius.com/artists/Cardi-b",
    "https://genius.com/artists/Mac-miller"
]

rappers = ["Eminem", "J. Cole", "Kanye West", "2Pac", 
           "Notorious B.I.G.", "Logic", "Nas", "Joyner Lucas", 
           "Juice WRLD", "Lil Pump", 
           "Nicki Minaj", "Cardi B", "Mac Miller"]
url = URLS[0]

In [4]:
driver.get(url)
div_btn = driver.find_element_by_xpath("//*[text()[contains(.,'Show all albums by')]]")

In [5]:
driver.get(url)
div_btn = driver.find_element_by_xpath("//*[text()[contains(.,'Show all albums by')]]")
page_source = driver.page_source

In [6]:
from collections import defaultdict
artists_dict = {}
failed = defaultdict(lambda: [])
for i, url in tqdm(enumerate(URLS)):
    lyrics = {}
    driver.get(url)
    div_btn = driver.find_element_by_xpath("//*[text()[contains(.,'Show all albums by')]]")
    driver.execute_script("arguments[0].click();", div_btn)
    time.sleep(1)

    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    scrollable = soup.find("scrollable-data")
    albums = soup.find_all('mini-album-card')
    for a in albums:
        links = a.find('a')
        href = links['href']
        title = links['title']

        page = requests.get(href).text
        soup = bs(page, 'lxml')
        tracks = soup.find_all("a", class_="u-display_block")
        album = {}
        album[href] = []
        for track in tqdm(tracks):
            track_href = track['href']
            title = track.find('h3').text.split('\n')[1].strip()
            reload = True
            count = 0
            while reload:
                if count == 20:
                    break 
                page = requests.get(track_href).text
                soup = bs(page, 'lxml')
                lyric_div = soup.find_all("div", class_="lyrics")
                reload = len(lyric_div) < 1
                count += 1
                
            if count == 20:
                album[href].append(track_href)
                failed[rappers[i]].append(album)
                continue

            text = lyric_div[0].text.split('\n')
            lyrics[title] = text
    artists_dict[rappers[i]] = lyrics

In [7]:
for r in rappers:
    with open("lyrics/" + r + ".pkl", "wb") as file:
        pickle.dump(artists_dict[r], file)

In [ ]:
list(failed.keys()), failed["Juice WRLD"]

In [9]:
data = {}
for r in rappers:
    with open("lyrics/" + r + ".pkl", "rb") as file:
        data[r] = pickle.load(file)

In [10]:
data.keys()

dict_keys(['Eminem', 'J. Cole', 'Kanye West', '2Pac', 'Notorious B.I.G.', 'Logic', 'Nas', 'Joyner Lucas', 'Juice WRLD', 'Lil Pump', 'Nicki Minaj', 'Cardi B', 'Mac Miller'])

In [11]:
data['2Pac']

{'TS11*': ['',
  '          ',
  '            Lyrics for this song have yet to be released. Please check back once the song has been released.',
  '          ',
  '        '],
 '2 of Amerikaz Most Wanted (Ft.\xa0Snoop\xa0Dogg)': ['',
  '',
  '[Intro: 2Pac & Snoop Doggy Dogg]',
  'Up out of there',
  "Ain't nothin' but a gangsta party",
  "Eh, light that up, Snoop! Why you actin' like that?",
  'Ahh shit, you done fucked up now',
  "(Ain't nothin' but a gangsta party)",
  "You done put two of America's most wanted",
  "In the same motherfuckin' place at the same motherfuckin' time?",
  "(Ain't nothin' but a gangsta party)",
  "Ha, ha, ha, ha, y'all niggas about to feel this",
  "(Ain't nothin' but a gangsta party)",
  "Break out the champagne glasses and the motherfuckin' condoms",
  "Have one on us, a'ight?",
  "(Ain't nothin' but a gangsta party)",
  'A toast to the gangsters',
  '',
  '[Verse 1: 2Pac]',
  'Picture perfect, I paint a perfect picture',
  "Bomb the hoochies with precisi

In [12]:
del data['2Pac']['TS11*']

## Cleaning the data

In [13]:
lyrics = {}
for k, v in data.items():
    lyrics[k] = []
    for t in v:
        lyrics[k].append(' '.join(v[t]))

for k in lyrics:
    combined_text = ' '.join(lyrics[k])
    lyrics[k] = [combined_text]
    
## key: rapper, value: all lyrics in one string

In [14]:
import pandas as pd
pd.set_option('max_colwidth', 150)

data_df = pd.DataFrame.from_dict(lyrics).transpose()
data_df.columns = ['lyrics']
data_df = data_df.sort_index()
data_df

,lyrics
2Pac,"[Intro: 2Pac & Snoop Doggy Dogg] Up out of there Ain't nothin' but a gangsta party Eh, light that up, Snoop! Why you actin' like that? Ahh shit,..."
Cardi B,"[Intro: Cardi B, Al ""T"" McLaran & Megan Thee Stallion] Whores in this house There's some whores in this house There's some whores in this house ..."
Eminem,"[Intro: Eminem] Yeah So I guess this is what it is, huh? Think it's obvious We ain't never gonna see eye to eye But it's funny As much as I hate..."
J. Cole,"[Intro] Are you doin' this work to facilitate growth or to become famous? Which is more important? Getting or letting go? You can do anything, a..."
Joyner Lucas,"[Intro: Ashanti] Fall slowly Fall slowly [Verse 1: Joyner Lucas] I love you more than I love myself If I weren't with you, I can't picture me w..."
Juice WRLD,"[Intro] Nah-nah-nah-nah-nah-nah Oh, oh [Pre-Chorus] Smoke cigarettes, cancer in the chest like cardiac arrest I've been feelin' stressed, tryna..."
Kanye West,"[Chorus] Sing every hour (Every hour, 'til the power) Every minute (Every minute, of the Lord) Every second (Every second, comes) Sing each and ..."
Lil Pump,"Lyrics from first snippet [Lil Pump] ...Elliot dinner (Brr), man, fuck Ben Baller (Ayy) VVS diamond said he got my dawg callin' (Brr, huh?)... ..."
Logic,Lyrics for this song have yet to be released. Please check back once the song has been released. [In...
Mac Miller,"[Intro: Telly & Mac Miller] When you're young, not much matters When you find something that you care about, then that's all you got When you go..."


In [15]:
data_df.lyrics.loc['2Pac']

'  [Intro: 2Pac & Snoop Doggy Dogg] Up out of there Ain\'t nothin\' but a gangsta party Eh, light that up, Snoop! Why you actin\' like that? Ahh shit, you done fucked up now (Ain\'t nothin\' but a gangsta party) You done put two of America\'s most wanted In the same motherfuckin\' place at the same motherfuckin\' time? (Ain\'t nothin\' but a gangsta party) Ha, ha, ha, ha, y\'all niggas about to feel this (Ain\'t nothin\' but a gangsta party) Break out the champagne glasses and the motherfuckin\' condoms Have one on us, a\'ight? (Ain\'t nothin\' but a gangsta party) A toast to the gangsters  [Verse 1: 2Pac] Picture perfect, I paint a perfect picture Bomb the hoochies with precision, my intention\'s to get richer With the S-N double-O-P, Dogg, my fuckin\' homie You\'s a cold-ass nigga on them hogs  [Verse 2: Snoop Doggy Dogg] Sure \'nough, I keep my hand on my gun \'Cause they got me on the run Now I\'m back in the courtroom, waitin\' on the outcome "Free 2Pac" is all that\'s on a nigga\

## Data Cleaning 1:
- Make text all lower case
- punctuaction marks
- remove numbers

In [16]:
import re
import string

def clean_text1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

clean1 = lambda x: clean_text1(x)

In [17]:
data_cleaned = pd.DataFrame(data_df.lyrics.apply(clean1))

In [18]:
data_cleaned

,lyrics
2Pac,up out of there aint nothin but a gangsta party eh light that up snoop why you actin like that ahh shit you done fucked up now aint nothin but ...
Cardi B,whores in this house theres some whores in this house theres some whores in this house theres some whores in this house hol up i said certified...
Eminem,yeah so i guess this is what it is huh think its obvious we aint never gonna see eye to eye but its funny as much as i hate you i need you this...
J. Cole,are you doin this work to facilitate growth or to become famous which is more important getting or letting go you can do anything anything you ...
Joyner Lucas,fall slowly fall slowly i love you more than i love myself if i werent with you i cant picture me with someone else and every time we separat...
Juice WRLD,nahnahnahnahnahnah oh oh smoke cigarettes cancer in the chest like cardiac arrest ive been feelin stressed tryna find ways to impress her sho...
Kanye West,sing every hour every hour til the power every minute every minute of the lord every second every second comes sing each and every millisecond ...
Lil Pump,lyrics from first snippet elliot dinner brr man fuck ben baller ayy vvs diamond said he got my dawg callin brr huh lyrics from second snippet...
Logic,lyrics for this song have yet to be released please check back once the song has been released welc...
Mac Miller,when youre young not much matters when you find something that you care about then thats all you got when you go to sleep at night you dream of...


## Data Cleaning 2:
- remove nonsensical text e.g. "\n"

In [19]:
def clean_text2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

clean2 = lambda x: clean_text2(x)

In [20]:
data_cleaned = pd.DataFrame(data_cleaned.lyrics.apply(clean2))

In [21]:
data_cleaned

,lyrics
2Pac,up out of there aint nothin but a gangsta party eh light that up snoop why you actin like that ahh shit you done fucked up now aint nothin but ...
Cardi B,whores in this house theres some whores in this house theres some whores in this house theres some whores in this house hol up i said certified...
Eminem,yeah so i guess this is what it is huh think its obvious we aint never gonna see eye to eye but its funny as much as i hate you i need you this...
J. Cole,are you doin this work to facilitate growth or to become famous which is more important getting or letting go you can do anything anything you ...
Joyner Lucas,fall slowly fall slowly i love you more than i love myself if i werent with you i cant picture me with someone else and every time we separat...
Juice WRLD,nahnahnahnahnahnah oh oh smoke cigarettes cancer in the chest like cardiac arrest ive been feelin stressed tryna find ways to impress her sho...
Kanye West,sing every hour every hour til the power every minute every minute of the lord every second every second comes sing each and every millisecond ...
Lil Pump,lyrics from first snippet elliot dinner brr man fuck ben baller ayy vvs diamond said he got my dawg callin brr huh lyrics from second snippet...
Logic,lyrics for this song have yet to be released please check back once the song has been released welc...
Mac Miller,when youre young not much matters when you find something that you care about then thats all you got when you go to sleep at night you dream of...


In [22]:
data_df.to_pickle("corpus.pkl")

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

# convert to document term matrix
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_cleaned.lyrics)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_cleaned.index

In [24]:
data_dtm

,aa,aaaaaaaaaa,aaaaaaack,aaaaah,aaaaahhh,aaaaahhhh,aaaaayyyyooooo,aaaagain,aaaah,aaaahh,...,世界中で聴いてる,帰っていただいて結構,彼の行動が気になって仕方ないはず,感謝しています,最高だったでしょう,本当はロジックを愛してやまないんでしょう,楽しんでいただけたことを願っています,毎日,私たちは共に歴史を刻んできた,耳を塞ぐか
2Pac,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cardi B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Eminem,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
J. Cole,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Joyner Lucas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Juice WRLD,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
Kanye West,1,0,0,0,1,1,0,0,3,1,...,0,0,0,0,0,0,0,0,0,0
Lil Pump,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Logic,1,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Mac Miller,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
data_dtm.to_pickle("dtm.pkl")


In [28]:
data_cleaned.to_pickle('data_clean.pkl')
pickle.dump(cv, open('cv.pkl', 'wb'))